<a href="https://colab.research.google.com/github/dioschuarz/data_science/blob/main/llm/Tabular_Data_Retrieval_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install langchain --quiet \
             accelerate --quiet \
             huggingface_hub --quiet \
             sentence_transformers --quiet\
             faiss-gpu --quiet \
             faiss-cpu --quiet \
             einops --quiet \
             unidecode --quiet

In [ ]:
from accelerate import Accelerator
from langchain.chains import RetrievalQA, question_answering, ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader, UnstructuredCSVLoader
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain import HuggingFaceHub
import pandas as pd
from unidecode import unidecode
import os, re, requests, torch, zipfile

# Functions

In [ ]:
def get_zip_file(url):

  download = requests.get(url)

  with open(url.split('/')[-1], 'wb') as arq:
    arq.write(download.content)

  arq_zip = zipfile.ZipFile(url.split('/')[-1])

  return arq_zip

def extract_field_description(txt):

    field_pattern = re.compile(r"Campo:\s+(\w+)")
    description_pattern = re.compile(r"Descrição\s+:\s+(.+)")

    matches = field_pattern.findall(txt)
    descriptions = description_pattern.findall(txt)

    if len(matches) != len(descriptions):
        raise ValueError("Número de campos não corresponde ao número de descrições")

    data = {"Campo": matches, "Descrição": [description.replace("\r", "") for description in descriptions]}
    df = pd.DataFrame(data)

    return df

def get_answer(qachain, query):

  answer = qachain({"query": query})

  return answer['result'].strip().split('Question:')[0].split('Answer:')[0]

# Read Data

In [ ]:
ano = '2023'
mes = '07'
# Dicionário de dados
dict_url = 'https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/META/meta_lamina_fi_txt.zip'
# Lâmina
lam_url = f'https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_{ano}{mes}.zip'
# Informativo diário
inf_url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diairio_fi_{ano}{mes}.zip'

dict_zip = get_zip_file(dict_url)
lam_zip = get_zip_file(lam_url)

In [ ]:
with dict_zip as zip_ref:
    with zip_ref.open(dict_zip.namelist()[0]) as file:
        file_contents = file.read().decode(encoding = 'ISO-8859-1')

field_description_dict = extract_field_description(file_contents)
#field_description_dict = field_description_dict.applymap(lambda x: unidecode(x))

field_description_dict

,Campo,Descrição
0,ANO_ANTER_EXEMPLO,Ano anterior ao de referência (exemplo para co...
1,ANO_EXEMPLO,Ano de referência (exemplo para comparar custo...
2,ANO_SEM_RENTAB,Anos em que não foram apresentados dados de re...
3,CALC_RENTAB_FUNDO,Fórmula de cálculo de sua rentabilidade
4,CALC_RENTAB_FUNDO_GATILHO,Descrição da fórmula de cálculo da rentabilida...
...,...,...
71,VL_RETORNO_3ANO,Retorno bruto hipotético para 3 anos após dedu...
72,VL_RETORNO_5ANO,Retorno bruto hipotético para 5 anos após dedu...
73,VL_TAXA_ENTR_EXEMPLO,Valor da taxa de ingresso (exemplo para compar...
74,VL_TAXA_SAIDA_EXEMPLO,Valor da taxa de saída (exemplo para comparar ...


In [ ]:
dados_fundos = pd.read_csv(lam_zip.open(lam_zip.namelist()[0]), sep=';', encoding = 'ISO-8859-1',  on_bad_lines='warn')

Skipping line 2193: expected 76 fields, saw 113
Skipping line 2789: expected 76 fields, saw 139
Skipping line 2897: expected 76 fields, saw 142



In [ ]:
# Vamos criar um dicionário de tradução usando o 'field_description_df'
translation_dict = dict(zip(field_description_dict["Campo"], field_description_dict["Descrição"]))

# Renomear as colunas do DataFrame 'dados_fundos' usando o dicionário de tradução
dados_fundos.rename(columns=translation_dict, inplace=True)

# Agora o DataFrame 'dados_fundos' tem colunas traduzidas
dados_fundos = dados_fundos.fillna('Não aplicável')
dados_fundos = dados_fundos.replace("N/D", "Não aplicável").replace("N/A", "Não aplicável")
#dados_fundos = dados_fundos.applymap(lambda x: unidecode(x) if type(x) not in (float, int) else x)

dados_fundos.head()

,CNPJ do fundo,Denominação Social,Data de competência do documento,Nome fantasia do fundo,Endereço eletrônico,Público-alvo,Restrições de investimento,Objetivos do fundo,Política de investimento,Percentual do PL que pode ser aplicado em ativos do exterior,...,Despesas previstas para 3 anos (se a taxa total de despesas se mantiver constante),Despesas previstas para 5 anos (se a taxa total de despesas se mantiver constante),"Retorno bruto hipotético para 3 anos após dedução das despesas e do valor do investimento original (antes da incidência de impostos, de taxas de ingresso e/ou saída, ou de taxa de performance)","Retorno bruto hipotético para 5 anos após dedução das despesas e do valor do investimento original (antes da incidência de impostos, de taxas de ingresso e/ou saída, ou de taxa de performance)",Descrição da forma de remuneração dos distribuidores,"Informa se o principal distribuidor oferta, para o público alvo do fundo, preponderantemente fundos geridos por um único gestor, ou por gestores ligados a um mesmo grupo econômico",Informa se existe conflito de interesses no esforço de venda,Telefone do serviço de atendimento ao cotista,Endereço eletrônico e demais canais disponíveis para o encaminhamento de reclamações,Domínio :
0,00.068.305/0001-35,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2023-07-31,FIC EMPREENDER RF LP,www.caixa.gov.br,Investidores que buscam retorno por meio de in...,o FUNDO destina-se a acolher investimentos de ...,Proporcionar rentabilidade por meio da aplicaç...,Investir no mínimo 95% do patrimônio líquido d...,0.0,...,50.42,93.0,280.58,517.51,O serviço de distribuição de cotas de fundos d...,O distribuidor oferta ao cotista e potenciais ...,Não se aplica,8007260101,www.caixa.gov.br,Não aplicável
1,00.071.477/0001-68,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...,2023-07-31,BB Renda Fixa Automático Empresa Simples,www.bb.com.br,O fundo é destinado a investidores que pretend...,Não aplicável,O FUNDO tem como objetivo proporcionar a renta...,"Para alcançar seus objetivos, o FUNDO aplicará...",0.0,...,0.23,0.42,330.77,610.09,O Distribuidor fará jus a uma importância corr...,O principal Distribuidor do fundo é o Banco do...,"O Administrador, o Gestor ou partes a eles rel...",8007293886,"bbdtvm@bb.com.br - Praça XV de Novembro 20, 3º...",Não aplicável
2,00.073.041/0001-08,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...,2023-07-31,BB Besc Renda Fixa Pratico Credito Privado,www.bb.com.br/www.besc.com.br,O fundo é destinado a investidores que pretend...,Não aplicável,Acompanhar o CDI mediante aplicação em ativos ...,As aplicações do FUNDO deverão se subordinar a...,0.0,...,13.65,25.06,317.35,585.45,O Distribuidor fará jus a uma importância corr...,O principal Distribuidor do fundo é o Banco do...,"O Administrador, o Gestor ou partes a eles rel...",8007293886,"bbdtvm@bb.com.br - Praça XV de Novembro 20, 3º...",Não aplicável
3,00.089.915/0001-15,AMARIL FRANKLIN FUNDO DE INVESTIMENTO EM COTAS...,2023-07-31,FUNDO FRANKLIN,https://mercantildobrasil.com.br/BeneficiarioI...,Destinado a investidores pessoas físicas e jur...,Conforme dispuser o regulamento,objetivo buscar proporcionar aos seus cotistas...,As aplicações do FUNDO deverão estar represent...,20.0,...,54.67,100.45,102.96,175.84,A distribuição de cotas dos fundos administrad...,O Banco Mercantil do Brasil S.A. oferta ao púb...,Os fundos geridos e administrados pela Mercant...,3130576568,mb_corretora_gestao@mercantil.com.br,Não aplicável
4,00.180.995/0001-10,SAFRA EXECUTIVE MAX RF FUNDO DE INVESTIMENTO E...,2023-07-31,SAFRA EXECUTIVE MAX RENDA FIXA FUNDO DE INVEST...,www.safraasset.com.br,Destinado a investidores em geral que buscam u...,Não aplicável,Buscar um retorno adicional por meio de exposi...,Aplica em fundos de investimento com carteira ...,20.0,...,7.83,9.47,1323.17,1601.04,"A remuneração dos distribuidores, incluindo o ...","A distribuição é realizada, principalmente, pe...",Vide acima.,8007725755,www.safraasset.com.br,Não aplicável


In [ ]:
# Inicializar uma lista para armazenar os dados
docs_data = []

# Iterar sobre cada linha do DataFrame 'dados_fundos'
for index, row in dados_fundos.iterrows():
    cnpj = row["CNPJ do fundo"]

    # Concatenar todos os valores das colunas (exceto a primeira que é o CNPJ)
    text = ". ".join([f"{column_name} significa {value}" for column_name, value in row.items()])
    docs_data.append({"CNPJ do fundo": cnpj,
                      "Nome fantasia do fundo": row["Nome fantasia do fundo"],
                      "Denominação Social": row["Denominação Social"],
                      "text": text})

# Criar o DataFrame 'docs_fundos' a partir dos dados coletados
docs_fundos = pd.DataFrame(docs_data)
docs_fundos = docs_fundos.reset_index().rename(columns={'index':'page'})

# Visualizar o novo DataFrame
docs_fundos.head()

,page,CNPJ do fundo,Nome fantasia do fundo,Denominação Social,text
0,0,00.068.305/0001-35,FIC EMPREENDER RF LP,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,CNPJ do fundo significa 00.068.305/0001-35. De...
1,1,00.071.477/0001-68,BB Renda Fixa Automático Empresa Simples,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...,CNPJ do fundo significa 00.071.477/0001-68. De...
2,2,00.073.041/0001-08,BB Besc Renda Fixa Pratico Credito Privado,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...,CNPJ do fundo significa 00.073.041/0001-08. De...
3,3,00.089.915/0001-15,FUNDO FRANKLIN,AMARIL FRANKLIN FUNDO DE INVESTIMENTO EM COTAS...,CNPJ do fundo significa 00.089.915/0001-15. De...
4,4,00.180.995/0001-10,SAFRA EXECUTIVE MAX RENDA FIXA FUNDO DE INVEST...,SAFRA EXECUTIVE MAX RF FUNDO DE INVESTIMENTO E...,CNPJ do fundo significa 00.180.995/0001-10. De...


In [ ]:
print(docs_fundos[docs_fundos['CNPJ do fundo']=='05.100.221/0001-55'])

     page       CNPJ do fundo Nome fantasia do fundo  \
467   467  05.100.221/0001-55    BB Ações Small Caps   

                                    Denominação Social  \
467  BB AÇÕES SMALL CAPS FUNDO DE INVESTIMENTO EM C...   

                                                  text  
467  CNPJ do fundo significa 05.100.221/0001-55. De...  


# LLM Pre-Processing

In [ ]:
# loading data from the file
#loader = DataFrameLoader(docs_fundos,
loader = DataFrameLoader(docs_fundos[docs_fundos['CNPJ do fundo'].isin(['05.100.221/0001-55',
                                                                        '00.068.305/0001-35',
                                                                        '00.071.477/0001-68',
                                                                        '00.073.041/0001-08',
                                                                        '00.089.915/0001-15'])],
                         page_content_column="text")
data = loader.load()

splitter = CharacterTextSplitter(
  chunk_size=5000,
  chunk_overlap=50,
  separator=". "
)

data_split = splitter.split_documents(data)

# for i in loader.lazy_load():
#     print(i)

In [ ]:
#data[0].page_content

In [ ]:
#data_split[2]

In [ ]:
dados_fundos.shape

(5402, 76)

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ''

In [ ]:
accelerator = Accelerator()
device = accelerator.device
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from langchain.embeddings import HuggingFaceHubEmbeddings

model_name = "tiiuae/falcon-7b"
#model_name = "meta-llama/Llama-2-7b-hf"
#model_name = 'google/flan-t5-base'

# model_kwargs = {'device': device,
#                 'trust_remote_code': True}

# encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings()

embeddings = accelerator.prepare(embeddings)

In [ ]:
vectorStore = FAISS.from_documents(data_split,
#vectorStore = FAISS.from_documents(data,
                                   embeddings)

# Without Prompt

In [ ]:
llm = HuggingFaceHub(repo_id=model_name,
                     model_kwargs={"temperature":0.1,
                                   "top_k":10,
                                   "max_length":1024,
                                   "max_new_tokens":300,
                                   "min_new_tokens":50,
                                   "num_return_sequences":1})


In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=vectorStore.as_retriever(search_kwargs={'k': 1}),
                                    return_source_documents=True)

In [ ]:
# question =  "Qual o público alvo do BB AÇÕES SMALL CAPS?"

# chain({"query": question})

In [ ]:
question =  "Qual o público alvo do fundo BB ACOES SMALL CAPS?"

answer = get_answer(chain, question)
answer

'O fundo é destinado a investidores que pretendem: investir em ações de empresas de pequeno e médio porte e de grande potencial de crescimento no longo prazo e que sejam investidores clientes do Banco do Brasil.\n\n'

# With Prompt

In [ ]:
# Prepare Falcon Huggingface API
llm = HuggingFaceHub(repo_id=model_name,
            model_kwargs = {
                "max_length":512,
                "max_new_tokens":300,
                "min_new_tokens":50,
                "temperature":0.1,
                "repetition_penalty": 1.5,
                "top_k":1,
                "num_return_sequences":1
            }
      )

# prepare stuff prompt template
prompt_template = """
You are a talkative AI assistant. Use the question to search a answer in the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Answer all user questions using at maximum 500 characters.

Context: {context}

Question: {question}

Answer:
""".strip()

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

chain_type_kwargs = {"prompt" : prompt}

chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorStore.as_retriever(search_kwargs={'k': 1}),
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs
    )

In [ ]:
question =  "Qual o publico alvo do fundo BB ACOES SMALL CAPS?"

answer = get_answer(chain, question)
answer

'O fundo é destinado a investidores que pretendem: investir em ações de empresas de pequeno e médio porte e de grande potencial de crescimento no longo prazo e que sejam investidores clientes do Banco do Brasil.\n\n'

In [ ]:
question =  "Qual o publico alvo do fundo BB RENDA FIXA AUTOMATICO EMPRESA SIMPLES?"

answer = get_answer(chain, question)
print(answer)

O fundo é destinado a investidores que pretendem: O FUNDO destina-se a receber aplicações de pessoas jurídicas, clientes do Banco do Brasil S/A, que busquem acompanhar a variação do CDI/SELIC, com perfil de aplicação de curto prazo.




In [ ]:
question =  'Qual publico-alvo do fundo franklin?'

answer = get_answer(chain, question)
print(answer)

Destinado a investidores pessoas físicas e jurídicas em geral, inclusive por meio de fundos de investimento, fundos de investimento em cotas de fundos de investimento.


